## 🙋‍♂️ Welcome to the TMY weekly solar data aggregator ☀️

This notebook runs an ETL procedure on solar data from various locations in the USA.

### Data source
__Source link:__ [tmy3-solar](https://www.kaggle.com/datasets/us-doe/tmy3-solar/code)

__File:__ `tmy3.csv`  
* __Description:__ One Year of typical Hourly Solar & Weather Data for +1000 US Locations
* __Data points:__ 
    * Date (MM/DD/YYYY)
    * Time (HH:MM)
    * GHI (W/m^2)
    * DNI (W/m^2)
    * station_number

__File:__ `TMY3_StationsMeta.csv`  
* __Description:__ Metadata about weather stations
* __Data points:__ 
    * USAF
    * Site Name
    * Latitude
    * Longitude

__File:__ `43256.pdf`
* __Description:__ User Guide for TMY3 data


### Transformations
This code aggregates the hourly GHI and DNI values in `tmy3.csv` into weekly averages for each station.

### Outputs
A .json file at a specified path.



## ⚙️ Set up
Ensure you have a Kaggle API access key at: `C:\Users\<your_name>\.kaggle\kaggle.json`  

Check the README for further info on installing required packages if necessary.

In [ ]:
# uncomment to create a conda environment 'solar' with the required packages
# !conda env create -f environment.yml

In [ ]:
# import packages
import pandas as pd
import opendatasets as od 
import json
from datetime import timedelta

# specify the output filepath
output_path = 'output.json'

## ⬇️ Import data

In [ ]:
kaggle_data = "https://www.kaggle.com/datasets/us-doe/tmy3-solar"
od.download_kaggle_dataset(dataset_url=kaggle_data, data_dir="./data")

In [ ]:
# read csv into a dataframe
tmy3 = pd.read_csv(r"data\tmy3-solar\tmy3.csv", usecols=['Date (MM/DD/YYYY)', 'Time (HH:MM)', 'GHI (W/m^2)', 'DNI (W/m^2)', 'station_number'])
tmy3.columns = ['date', 'time', 'ghi', 'dni', 'station']
tmy3.head()

In [ ]:
# import metadata
tmy3_meta = pd.read_csv(r"data\tmy3-solar\TMY3_StationsMeta.csv")
tmy3_meta.set_index('USAF', inplace=True)
tmy3_meta.head()

## 🧹 Cleaning

### __NaN__
There are a few NaN values in the dataset.  These should be dropped, as they represent missing data and may distort aggregations.

In [ ]:
# there are a few NA values.  These should be dropped as they represent missing data.
tmy3.isnull().sum()

In [ ]:
tmy3.dropna(inplace=True)
tmy3

#### __24:00__
The source csv file includes 24-hour data with the value '24:00' indicating midnight. The ISO 8601 standard uses 00:00 for midnight and does not use 24:00.  Therefore, rows in the dataset with a time value of 24:00 will need to be changed to 00:00.  The date of these rows will also need to increment by 1. 

__Example__  
Incorrect: 2000-01-01T24:00:00  
Correct: 20000-01-02T00:00:00

Therefore this cleaning step will replace some time values and increment the date of those values.

This step also creates a 'timestamp' column with pandas datetime objects to use as an index.  This will assist with date operations and aggregating.

In [ ]:
# identify items with time = 24:00 and change to 00:00
mask_24 = tmy3['time'] == '24:00'
tmy3.loc[mask_24, 'time'] = '00:00'

# create 'timestamp' column with pandas datetime objects
tmy3['timestamp'] = pd.to_datetime(tmy3['date'] + ' ' + tmy3['time'], format='%m/%d/%Y %H:%M')

# increment the 'timestamp' column by one for items that were changed from 24:00 to 00:00
tmy3.loc[mask_24, 'timestamp'] += timedelta(days=1)

# drop the 'date' and 'time' columns, as they are no longer needed.  The 'date' column is also incorrect now.
tmy3.drop(['date', 'time'], axis=1, inplace=True)

# set the index as the 'timestamp' column, which is required for further processing steps
tmy3.set_index('timestamp', inplace=True)

# inspect the data
tmy3.head()

## 🦩 Wrangling

__NB:__ 
When you use `pandas.resample('W')` to resample data by week, the following rules apply:

1. Included Timestamps in Each Week  

    __Weekly Grouping:__ `resample('W')` groups data from Monday through Sunday. Each group includes all timestamps from the beginning of Monday at 00:00:00 to the end of Sunday at 23:59:59.

    __Resulting Timestamp__: The resulting timestamp for each week is set to the end of that week, which is Sunday.

    __Non-Sunday Starting Weeks:__ If your data does not start on a Monday, the first "partial week" will still include data from the first available day up to that Saturday.

2. Customizing the Week Ending Day  

    You can change the default behavior of grouping weeks from Sunday to Saturday by using `resample('W-MON')`, `resample('W-FRI')`, etc., where the argument specifies which day of the week the resampling should end on:

    `resample('W-MON')`: Groups from Tuesday to Monday, with the resulting timestamp set to Monday.  
    `resample('W-FRI')`: Groups from Saturday to Friday, with the resulting timestamp set to Friday.  

_(source: ChatGPT)_

In [ ]:
value = 'ghi'
station = 690150
week_end = pd.to_datetime('1996-08-11 23:00')
week_start = week_end - timedelta(days=7)

# check a value in the existing df to validate the transform
df = tmy3.query("station == @station and @week_start < timestamp <= @week_end ")
print(f"Mean '{value}' at station {station} for week ending {week_end} is: \n{df[value].mean()}")

In [ ]:
def wrangle(df: pd.DataFrame):
    """
    Perform transformations on the Dataframe object. This function aggregates by station, and then
    resamples the 'ghi' and 'dni' values to a weekly mean.
    """
    output = df.groupby('station').resample('W').mean(['ghi', 'dni']).drop('station', axis=1).fillna(0)
    print('Data has been transformed.')
    return output

tmy3_transform = wrangle(tmy3)
tmy3_transform.query("station == @station")

## ⬆️ Outputs

In [ ]:
json_output = [] 

for station, group in tmy3_transform.groupby('station'):
    # get site information from the metadata df
    site_name = tmy3_meta.loc[station, 'Site Name']
    coordinates = [tmy3_meta.loc[station, 'Latitude'], tmy3_meta.loc[station, 'Longitude']]
    
    # compile the data points for each station
    data_list = []
    for timestamp, row in group.iterrows():
        data_entry = {
            # get the timestamp in milliseconds since epoch
            'timestamp': int(timestamp[1].timestamp() * 1000), 
            'ghi': row['ghi'],
            'dni': row['dni']
        }
        data_list.append(data_entry)

    # compile the full json for each station
    json_entry = {
        'id': station,
        'site_name': site_name,
        'coordinates': coordinates, 
        'data': data_list
    }

    json_output.append(json_entry)

# output to file
with open(output_path, 'w') as file:
    json.dump(json_output, file, indent=4)

print(f'Data has been saved to {output_path}')

# inspect the output
print(json.dumps(json_output, indent=4, default=str))
